In [1]:
#package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import re
import cufflinks
from nltk.corpus import stopwords
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')






# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical


In [ ]:
# data = pd.read_csv("NDA.csv")
# NDA = data.copy()
# NDA = NDA.dropna(how = 'all')

In [ ]:
NDA['Category'].value_counts().sort_values(ascending = False).iplot(kind = "bar", 
yTitle = 'Number of class', title = 'Number of cases in each category')

In [ ]:
'''a function which re-catogerize the NDA clauses in order to improve model accuracy and make the 
classification easier
Originally there are 17 categories, and now we are compressing it to 7'''

def reclassify(row):
    if row['Category'] == 1 or row['Category'] == 3:
        return 'DEF'
    elif row['Category'] == 2:
        return 'RIG'
    elif row['Category'] == 4 or row['Category'] == 15:
        return 'EXP'
    elif row['Category'] == 5 or row['Category'] == 6:
        return 'WAR'
    elif row['Category'] == 7 or row['Category'] == 10:
        return 'GOV'
    elif row['Category'] == 8:
        return 'REM'
    elif row['Category'] == 13 or row['Category'] == 16:
        return 'TER'
    else:
        return 'GEN'


NDA['CAT'] = NDA.apply(lambda row: reclassify(row), axis = 1)


In [ ]:
#the value counts
NDA['CAT'].value_counts()
#after reclassified
NDA['CAT'].value_counts().sort_values(ascending = False).iplot(kind = "bar", 
yTitle = 'Number of class', title = 'Number of cases in each category after reclassification')

In [ ]:
#a process of cleaning data 
NDA = NDA.reset_index(drop=True)
REPLACE_BY_SPACE = re.compile('[/(){}\[\]\|@,;\n\r*]')
BAD_SYMBOLS = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(para):

    para = str(para)
    para = para.lower()
    para = REPLACE_BY_SPACE.sub(' ', para)
    para = BAD_SYMBOLS.sub('', para)
    para = ' '.join(word for word in para.split() if word not in STOPWORDS)
    para = para.lstrip('0123456789.- ') #remove the digits/space/dots at the start of the string

    return para


NDA['cleaned_txt'] = NDA['Text'].apply(clean_text) #the cleaned data
    